In [10]:
import os
import csv
import json
import urllib.request

def download_json(url):
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf-8')
    return data

def save_text_file(file_name, content):
    with open(file_name, 'w') as file:
        file.write(content)

# Read the list of filenames
file_list_path = 'C:/Users/GnTRnD/Desktop/gen_ai/zero_city/fill_anything/OTH/oth.txt'
file_list = []
with open(file_list_path, 'r') as file:
    for line in file:
        file_list.append(line.strip())

# Read the CSV file
csv_file = 'track.csv'
json_urls = []

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['up_file_nm'] in file_list:
            json_urls.append((row['low_file_nm'], row['low_file_url']))

output_path = 'C:/Users/GnTRnD/Desktop/gen_ai/zero_city/fill_anything/OTH/labels'

os.mkdirs('./labels', exist_ok=True)

# Download JSON files and save content as text
for file_nm, url in json_urls:
    try:
        json_data = download_json(url)
        data = json.loads(json_data)

        # Extract the annotations
        annotations = data['annotations']

        # Remove the .json extension from file_nm
        file_nm = file_nm.split('.json')[0]

        # Create the text file and write the information
        output_file = os.path.join(output_path, f'{file_nm}.txt')  # Use file_nm as the file name
        with open(output_file, 'w') as f:
            for annotation in annotations:
                label_id = annotation['label_id']
                image_bbox = ' '.join(str(value) for value in annotation['info'])
                f.write(f'{label_id} {image_bbox}\n')
    except FileNotFoundError:
        # File does not exist, so skip it
        continue
